In [1]:
import numpy as np
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader, TensorDataset
import matplotlib.pyplot as plt
from mpl_toolkits import mplot3d
import pandas as pd
from sklearn.model_selection import LeaveOneOut, KFold
from binaryapproximator import *
from sklearn.metrics import zero_one_loss, log_loss
import optuna

/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [7]:
octet = pd.read_csv('octet.csv')
octet_selected = octet[['sign(Delta E)', 'Delta E', 'IP(A)', 'EA(A)', 'H(A)', 'L(A)', 'rs(A)', 'rp(A)', 'rd(A)', 'IP(B)', 'EA(B)', 'H(B)', 'L(B)', 'rs(B)', 'rp(B)', 'rd(B)', 'EN(A)', 'EN(B)']]
X = octet_selected[['IP(A)', 'EA(A)', 'H(A)', 'L(A)', 'rs(A)', 'rp(A)', 'rd(A)', 'IP(B)', 'EA(B)', 'H(B)', 'L(B)', 'rs(B)', 'rp(B)', 'rd(B)', 'EN(A)', 'EN(B)']].to_numpy()
# y_delta = octet_selected[['Delta E']].to_numpy()
y = octet_selected[['sign(Delta E)']].to_numpy()
y = np.where( y==1, np.float32(1), np.float32(0))

In [3]:
input = 16
hidden1 = 32
hidden2 = 32
hidden3 = 32
output = 1
loss_criterion = nn.BCELoss()
loo_cv = LeaveOneOut()

# Scenario 0: Regular NN Training
1. Initialise the Weights
    
For each loop:

2. $y_{pred} = Model(X,W_t, \theta_t)$ (Froward Step)
3. $loss(y, y_{pred})$ 
4. $\nabla_{w_t} = \frac{\partial loss}{\partial W_t}$ and $\nabla_{\theta_t} = \frac{\partial loss}{\partial \theta_t}$ (Backward Step)
5. $W_{t+1} = W_t + \eta \nabla_{w_t}$ and $\theta_{t+1} = \theta_t + \eta \nabla_{\theta_t}$ (Update Step)

In [16]:
def objective(trial):
    # Define the search space
    hidden_size1 = trial.suggest_int('hidden_size1', 2, 32)
    hidden_size2 = trial.suggest_int('hidden_size2', 2, 32)
    hidden_size3 = trial.suggest_int('hidden_size3', 2, 32)

    model_base_c = GeneralLogisticNN(
        input_size=input,
        hidden_size1=hidden_size1,
        hidden_size2=hidden_size2,
        hidden_size3=hidden_size3,
        output_size=output
    )

    tr = TrainNN(model=model_base_c, x=X, y=y, gradient_threshold=0.0001, learning_rate=0.001)
    loss_func = LossFunction(bce=True)
    model_final, epoch_loss = tr.nn_training(loss_func=loss_func)
    y_pred = model_final.predict(X)
    y_pred = np.where(y_pred>=0.5, 1, 0)
    loss = zero_one_loss(y,y_pred)

    return loss

# Create an Optuna study
study = optuna.create_study(direction='minimize')

# Start the optimization process
study.optimize(objective, n_trials=100)

# Print the best hyperparameters and metric
print('Best hyperparameters:', study.best_params)
print('Best metric value:', study.best_value)

[I 2023-11-28 15:18:46,483] A new study created in memory with name: no-name-4c21df1f-06cb-41d4-9a35-9f386f237264
[I 2023-11-28 15:18:50,109] Trial 0 finished with value: 0.0 and parameters: {'hidden_size1': 12, 'hidden_size2': 21, 'hidden_size3': 30}. Best is trial 0 with value: 0.0.


Early stopping at Epoch 6346 due to small gradients.


[I 2023-11-28 15:18:54,094] Trial 1 finished with value: 0.0 and parameters: {'hidden_size1': 14, 'hidden_size2': 24, 'hidden_size3': 31}. Best is trial 0 with value: 0.0.


Early stopping at Epoch 6901 due to small gradients.


[I 2023-11-28 15:18:57,978] Trial 2 finished with value: 0.0 and parameters: {'hidden_size1': 3, 'hidden_size2': 6, 'hidden_size3': 26}. Best is trial 0 with value: 0.0.


...

Early stopping at Epoch 7561 due to small gradients.


[I 2023-11-28 15:27:12,848] Trial 98 finished with value: 0.0 and parameters: {'hidden_size1': 16, 'hidden_size2': 8, 'hidden_size3': 14}. Best is trial 0 with value: 0.0.


Early stopping at Epoch 9269 due to small gradients.


[I 2023-11-28 15:27:16,751] Trial 99 finished with value: 0.0 and parameters: {'hidden_size1': 21, 'hidden_size2': 5, 'hidden_size3': 26}. Best is trial 0 with value: 0.0.


Early stopping at Epoch 6898 due to small gradients.
Best hyperparameters: {'hidden_size1': 12, 'hidden_size2': 21, 'hidden_size3': 30}
Best metric value: 0.0


In [17]:
def objective_binary(trial):
    # Define the search space
    hidden_size1 = trial.suggest_int('hidden_size1', 2, 32)
    hidden_size2 = trial.suggest_int('hidden_size2', 2, 32)
    hidden_size3 = trial.suggest_int('hidden_size3', 2, 32)

    model_base_c = GeneralLogisticNN(
        input_size=input,
        hidden_size1=hidden_size1,
        hidden_size2=hidden_size2,
        hidden_size3=hidden_size3,
        output_size=output
    )

    tr = BinaryTrainNN(model=model_base_c, x=X, y=y, gradient_threshold=0.0005, learning_rate=0.001)
    loss_func = LossFunction(bce=True)
    model_final, epoch_loss = tr.nn_training(loss_func=loss_func)
    model_final.binarise_model(layer=3)
    y_pred = model_final.predict_binary(X)
    y_pred = np.where(y_pred>=0.5, 1, 0)
    loss = zero_one_loss(y,y_pred)

    return loss

# Create an Optuna study
study = optuna.create_study(direction='minimize')

# Start the optimization process
study.optimize(objective, n_trials=100)

# Print the best hyperparameters and metric
print('Best hyperparameters:', study.best_params)
print('Best metric value:', study.best_value)

[I 2023-11-28 15:36:25,633] A new study created in memory with name: no-name-a3dab5be-84c1-4c01-93e6-0a73d7edd30c
[I 2023-11-28 15:36:29,608] Trial 0 finished with value: 0.0 and parameters: {'hidden_size1': 29, 'hidden_size2': 5, 'hidden_size3': 29}. Best is trial 0 with value: 0.0.


Early stopping at Epoch 6994 due to small gradients.


[I 2023-11-28 15:36:34,707] Trial 1 finished with value: 0.0 and parameters: {'hidden_size1': 18, 'hidden_size2': 11, 'hidden_size3': 12}. Best is trial 0 with value: 0.0.


Early stopping at Epoch 8944 due to small gradients.


[I 2023-11-28 15:36:39,119] Trial 2 finished with value: 0.0 and parameters: {'hidden_size1': 30, 'hidden_size2': 31, 'hidden_size3': 20}. Best is trial 0 with value: 0.0.


...

Early stopping at Epoch 8487 due to small gradients.


[I 2023-11-28 15:44:42,868] Trial 98 finished with value: 0.0 and parameters: {'hidden_size1': 30, 'hidden_size2': 5, 'hidden_size3': 23}. Best is trial 0 with value: 0.0.


Early stopping at Epoch 7392 due to small gradients.


[I 2023-11-28 15:44:46,903] Trial 99 finished with value: 0.0 and parameters: {'hidden_size1': 28, 'hidden_size2': 11, 'hidden_size3': 27}. Best is trial 0 with value: 0.0.


Early stopping at Epoch 6973 due to small gradients.
Best hyperparameters: {'hidden_size1': 29, 'hidden_size2': 5, 'hidden_size3': 29}
Best metric value: 0.0


In [ ]:
y = octet_selected[['Delta E']].to_numpy()
def objective_regression(trial):
    # Define the search space
    hidden_size1 = trial.suggest_categorical('hidden_size1', [4, 6, 8, 16, 32])
    hidden_size2 = trial.suggest_categorical('hidden_size2', [4, 6, 8, 16, 32])
    hidden_size3 = trial.suggest_categorical('hidden_size3', [4, 6, 8, 16])

    model_base_c =GeneralRegressionNN(
        input_size=input,
        hidden_size1=hidden_size1,
        hidden_size2=hidden_size2,
        hidden_size3=hidden_size3,
        output_size=output
    )

    tr = TrainNN(model=model_base_c, x=X, y=y, gradient_threshold=0.005, learning_rate=0.0001)
    loss_func = LossFunction(mse=True, l1_lambda=0.001)
    model_final, epoch_loss = tr.nn_training(loss_func=loss_func)
    y_pred = model_final.predict(X)
    # y_pred = np.where(y_pred>=0.5, 1, 0)
    loss = np.mean((y-y_pred)**2)

    return loss

# Create an Optuna study
study = optuna.create_study(direction='minimize')

# Start the optimization process
study.optimize(objective_regression, n_trials=50)

# Print the best hyperparameters and metric
print('Best hyperparameters:', study.best_params)
print('Best metric value:', study.best_value)